In [1]:
from collections import defaultdict

class Station:
    code_mapping = {}
    train_mapping = defaultdict(list)

    def __init__(self, code, name):
        self.code = code
        self.name = name

        Station.code_mapping[code] = self

        # Special case for shivaji terminus
        if code == "CSMT":
            Station.code_mapping["CSTM"] = self
            
    def __str__(self):
        return f"{self.code}: {self.name}"

In [2]:
class Train:
    mapping = {}

    def __init__(self, number, name, src: Station, dst: Station):
        self.number = number
        self.name = name
        self.src = src
        self.dst = dst
        self.stops = []
        Train.mapping[number] = self

    def add_stop(self, time, code):
        station = Station.code_mapping[code]
        Station.train_mapping[code].append(self)
        self.stops.append((time, station))

    def __str__(self):
        return f"Train {self.name} ({self.name}): {self.src} -> {self.dst} with {self.stops.__len__()} stops"

In [3]:
import pandas as pd

railway_data = pd.read_csv("./train_details_2017.csv")
railway_data = railway_data.dropna()

/tmp/ipykernel_109846/4281938988.py:3: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  railway_data = pd.read_csv("./train_details_2017.csv")


In [4]:
# Index stations
for index, row in railway_data.iterrows():
    station_code = row["Station Code"]
    station_name = row["Station Name"]

    # Create a Station object if it doesn't exist yet
    if station_code not in Station.code_mapping:
        Station(station_code, station_name)


In [5]:
# Index trains
for index, row in railway_data.iterrows():
    num = row["Train No"]
    name = row["Train Name"]

    train = None
    if num not in Train.mapping:
        src = Station.code_mapping[row["Source Station"]]
        dst = Station.code_mapping[row["Destination Station"]]
        train = Train(num, name, src, dst)
    else:
        train = Train.mapping[num]

    seq = row["SEQ"]
    if seq == 1:
        # add first stop with departure time
        time = row["Departure Time"]
        station = row["Station Code"]
        train.add_stop(time, station)
    else:
        # add other stops with arrival time
        time = row["Arrival time"]
        station = row["Station Code"]
        train.add_stop(time, station)

In [6]:
from datetime import timedelta
from datetime import datetime

def update_time(data):
    for i, val in enumerate(data):
        dt = datetime.strptime(val, '%H:%M:%S')
        if i > 0 and dt < prev_dt:
            days_diff = (prev_dt - dt).days + 1
            dt += timedelta(days=days_diff)
        data[i] = dt
        prev_dt = dt

data = ['21:46:00', '22:27:00', '23:51:00', '01:00:00', '01:30:00']
update_time(data)
print(data)

[datetime.datetime(1900, 1, 1, 21, 46), datetime.datetime(1900, 1, 1, 22, 27), datetime.datetime(1900, 1, 1, 23, 51), datetime.datetime(1900, 1, 2, 1, 0), datetime.datetime(1900, 1, 2, 1, 30)]


In [7]:
dt = datetime(1900, 1, 1, 21, 46)
dt.strftime("%H:%M:%S")

'21:46:00'

In [8]:
# Increment station times with additional 24 hrs where it crosses a day
for train in Train.mapping.values():
    times = [tup[0] for tup in train.stops]
    stops = [tup[1] for tup in train.stops]
    update_time(times)
    train.stops = list(zip(times, stops))

In [9]:
def query_night_sleeper_train_from_station(station_code):
    station = Station.code_mapping[station_code]
    trains = Station.train_mapping[station_code]

    return [(train, query_night_sleeper_train_destinations(train, station)) for train in trains if (query_night_sleeper_train(train, station))]

def query_night_sleeper_train(train: Train, station: Station) -> bool:
    # check that train starts from source station between 7pm-2am
    for (time, stop_station) in train.stops:
        if stop_station == station:
            time_string = time.strftime("%H:%M:%S")
            if ("19:00:00" <= time_string <= "23:59:59") or ("00:00:00" <= time_string <= "02:00:00"):
                return True
            
    return False

def query_night_sleeper_train_destinations(train: Train, station: Station) -> list[Station]:
    start_time = None
    for (time, stop_station) in train.stops:
        if stop_station == station:
            start_time = time

    stations = []
    lower = timedelta(hours=6)
    upper = timedelta(hours=12)
    for (time, stop_station) in train.stops:
        if time > start_time:
            if lower <= time - start_time <= upper:
                stations.append(stop_station)
            
    return stations

data = query_night_sleeper_train_from_station("HWH")
print(len(data))
for (train, dests) in data:
    print(f"Train: {train}")
    for dest in dests:
        print(f"{dest}")


171
Train: Train TVC-SCL SPL (TVC-SCL SPL): TVC: TRIVANDRUM C -> SCL: SILCHAR with 58 stops
MLDT: MALDA TOWN
KNE: KISHANGANJ
NJP: NEW JALPAIGU
DQG: DHUPGURI
Train: Train RNC HWH SHAT (RNC HWH SHAT): RNC: RANCHI -> HWH: HOWRAH JN. with 10 stops
Train: Train BARABIL-HOWR (BARABIL-HOWR): BBN: BARBIL -> HWH: HOWRAH JN. with 11 stops
Train: Train JNANESWARISU (JNANESWARISU): HWH: HOWRAH JN. -> LTT: LOKMANYA TIL with 17 stops
ROU: ROURKELA
JSG: JHARSUGUDA J
BSP: BILASPUR JN.
Train: Train HOWRAH PUNE (HOWRAH PUNE): HWH: HOWRAH JN. -> PUNE: PUNE JN. with 30 stops
ROU: ROURKELA
JSG: JHARSUGUDA J
BRJN: BRAJARAJNAGA
RIG: RAIGARH
CPH: CHAMPA JN.
BSP: BILASPUR JN.
Train: Train HWH-LTT SAMR (HWH-LTT SAMR): HWH: HOWRAH JN. -> LTT: LOKMANYA TIL with 23 stops
CKP: CHAKARADHARP
ROU: ROURKELA
JSG: JHARSUGUDA J
RIG: RAIGARH
Train: Train PA-HWH DURAN (PA-HWH DURAN): PUNE: PUNE JN. -> HWH: HOWRAH JN. with 8 stops
Train: Train YPR-BGP ANGA (YPR-BGP ANGA): YPR: YESVANTPUR J -> BGP: BHAGALPUR with 29 stops
KIU

In [3]:
# Do the same using sql
import pandas as pd
timetable = pd.read_csv("./train_details_2017.csv")

/tmp/ipykernel_1557920/82050991.py:3: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  timetable = pd.read_csv("./train_details_2017.csv")


In [6]:
# Filter evening trains
evening_trains = timetable[
    (timetable['Station Code'] == 'HWH') &
    (timetable['Departure Time'].between('19:00:00', '22:00:00'))
][['Train No', 'Departure Time']]

# Filter morning stations
morning_stations = timetable[
    timetable['Arrival time'].between('06:00:00', '10:00:00')
][['Train No', 'Station Code', 'Arrival time']]

# Merge evening trains and morning stations
trains_with_travel_time = evening_trains.merge(
    morning_stations, on='Train No', suffixes=('_start', '_end')
)


In [12]:
# Calculate travel time and add a day if needed
trains_with_travel_time['travel_time'] = (
    pd.to_timedelta(trains_with_travel_time['Arrival time']) -
    pd.to_timedelta(trains_with_travel_time['Departure Time'])
)
trains_with_travel_time.loc[
    trains_with_travel_time['travel_time'] < pd.Timedelta(0),
    'travel_time'
] += pd.Timedelta(days=1)

# Convert travel time to hours
trains_with_travel_time['travel_time'] /= pd.Timedelta(hours=1)

# Filter stations with travel time between 6 and 10 hours
result = trains_with_travel_time['Station Code'].drop_duplicates().sort_values()
# result = trains_with_travel_time[
#     trains_with_travel_time['travel_time'].between(6, 10)
# ]['Station Code'].drop_duplicates().sort_values()

# Print the result
print(len(result))

133


In [17]:
import pandas as pd
import sqlite3

# Read the CSV file
timetable = pd.read_csv('train_details_2017.csv')

# Create an in-memory SQLite database
connection = sqlite3.connect(':memory:')

# Write the pandas DataFrame to the SQLite database
timetable.to_sql('timetable', connection, if_exists='replace', index=False)


/tmp/ipykernel_1557920/3212819658.py:5: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  timetable = pd.read_csv('train_details_2017.csv')


186124

In [19]:
# Define the SQL query
sql_query = """
WITH evening_trains AS (
  SELECT "Train No", "Departure Time" AS start_time, SEQ AS start_seq
  FROM timetable
  WHERE "Station Code" = 'HWH'
    AND "Departure Time" BETWEEN '19:00:00' AND '22:00:00'
),
morning_stations AS (
  SELECT "Train No", "Station Code", "Arrival Time" AS end_time, SEQ AS end_seq
  FROM timetable
  WHERE "Arrival Time" BETWEEN '06:00:00' AND '10:00:00'
),
trains_with_travel_time AS (
  SELECT et."Train No", et.start_time, ms."Station Code", ms.end_time,
    (strftime('%s', ms.end_time) - strftime('%s', et.start_time) +
      (CASE WHEN ms.end_time < et.start_time THEN 86400 ELSE 0 END)) / 3600.0 AS travel_time
  FROM morning_stations ms
  JOIN evening_trains et ON ms."Train No" = et."Train No" and ms.end_seq > et.start_seq
)

SELECT DISTINCT "Station Code"
FROM trains_with_travel_time
WHERE travel_time BETWEEN 6 AND 10
ORDER BY "Station Code";
"""

# Execute the SQL query and store the result in a pandas DataFrame
result = pd.read_sql_query(sql_query, connection)

# Close the SQLite connection
connection.close()

# Print the result
print(result)

   Station Code
0           AKN
1          AUBR
2          BAND
3           BBU
4           BCA
5           BEH
6           BJU
7          BLQR
8           BMB
9           BSP
10          BUP
11          CMX
12          CPH
13         DKGS
14          DOS
15          DVL
16          DWO
17           GP
18          GPH
19          GZM
20         HLDD
21          JMP
22          JMQ
23          KIR
24          KTJ
25          KYN
26         MLDT
27          NFK
28           NK
29         PUNE
30            R
31         RAIR
32          RIG
33         RJPB
34          ROU
35         RUPC
36          SEB
37          SGG
38          SSM
